In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

import myfunctions
import myplot
import mylearn
import myhelpers

logitech_step1_fast = ('logitech-fast/niels-step_1-1617765899871', 0)
logitech_step1_slow = ('logitech-slow/niels-step_1-1617765658270', 60, True)
lenovo_slow = ('lenovo-slow-easy/niels-step_5-1617952576650', 0)
lenovo_slow2 = ('lenovo-slow-easy/niels-step_5-1618197371437', 0)
lenovo_hard = ('lenovo-hard/niels-step_1-1618199054005', 0)
noweb1 = ('winform/winform-1618904428438', 60, False)
noweb2 = ('winform/winform-1619147742756', 75, False)
noweb_logitech = ('winform/winform-1619400514590', -80, True) # Still lost some samples after about 100 seconds

def getData2(x, keystroke_min_peak_level):
    path, sync_adjustment, adjust_missing_samples = x
    d = myfunctions.getData(path,
                            sync_adjustment=sync_adjustment,
                            adjust_missing_samples=adjust_missing_samples,
                            keystroke_min_peak_level=keystroke_min_peak_level)
    print(myhelpers.getListGroupPercentages([x[0] for x in d['keystrokes']], '\n'))
    return d

In [ ]:
d = getData2(noweb_logitech, keystroke_min_peak_level=0.1)

In [ ]:
# myplot.plotLostSamples(d)

myplot.plotKeystrokeContext(d, 900, -80)

# for i in range(21, 31):
#     myplot.plotKeystroke(d, i)

# d.keys()

# set([x[3] for x in d['down_events']])

In [ ]:
# This worked pretty well (lenovo)
# mylearn.testClustering(myfunctions.getData('winform/winform-1619147742756',
#                                            sync_adjustment=75,
#                                            adjust_missing_samples=False,
#                                            keystroke_min_peak_level=0.1),
#     keep=None,
#     features=['mfcc_mean', 'mfcc_std'])

# Half-decent result using larger dataset (6 min, ~800 keystrokes), but still an easy case with just 4 keys (Return, Space, Tilde, Backspace)
mylearn.testClustering(myfunctions.getData('winform/winform-1619400514590',
                                           sync_adjustment=-80,
                                           adjust_missing_samples=False,
                                           keystroke_min_peak_level=0.1),
    keep=None,
    features=['mfcc_mean', 'mfcc_std'])

In [ ]:
mylearn.testClustering(d,
    keep=None,
    features=[
        'mfcc_mean',
        'mfcc_std',
#         'mfcc_max',
#         'mfcc_argmax_time',
#         'mfcc_argmax_channel'
    ])



In [ ]:
def findNumberOfClusters(features):
    x = list(range(2, 20))
    scores = []
    for n_clusters in tqdm(x):
        kmeans = sklearn.cluster.KMeans(n_clusters)
        kmeans.fit(features)
        kmeans_score = kmeans.score(features)
        clustering = kmeans.predict(features)
        kmeans_silhouette_score = silhouette_score(features, clustering)

        gm = sklearn.mixture.GaussianMixture(n_clusters)
        gm.fit(features)
        gm_score = gm.score(features)
        gm_bic_score = gm.bic(features)
        
        scores.append((kmeans_score, kmeans_silhouette_score, gm_score, gm_bic_score))
    return x, scores

features = mylearn.getConcatenatedFeatures(d, ['mfcc_max', 'mfcc_mean', 'mfcc_std'])
number_of_clusters,scores = findNumberOfClusters(mylearn.getConcatenatedFeatures(d, ['mfcc_max', 'mfcc_mean', 'mfcc_std']))

for (i, s) in enumerate(['kmeans_score', 'kmeans_silhouette_score', 'gm_score', 'gm_bic_score']):
    plt.figure()
    plt.title(s)
    plt.scatter(number_of_clusters, [x[i] for x in scores])
    plt.show()

In [ ]:
import librosa
sr = d['sr']
srms = int(sr/1000)
window_length = int(10*srms)
hop_length = int(2.5*srms)
wav = d['keystroke_wavs'][0]
print( len(wav) )
print( len(wav) / srms )
print( librosa.feature.spectral_centroid(wav, sr, win_length=window_length, hop_length=hop_length).shape )
print( librosa.feature.chroma_stft(wav, sr, win_length=window_length, hop_length=hop_length).shape )
print( librosa.feature.zero_crossing_rate(wav, frame_length=window_length, hop_length=hop_length).shape )

In [ ]:
mylearn.addFeatures(d)
print(d['normalised_mfcc_features'].shape)
print(d['mfcc_max'].shape)
print(d['mfcc_argmax_time'].shape)
print(d['mfcc_argmax_channel'].shape)
# import numpy as np
# print(np.argmax([[1,2,1], [3,4,5]], axis=0))

In [ ]:

for i in range(10, 17):
    myplot.plotKeystroke(d, i)

In [ ]:
def p(key):
    indexes = [i for i, k in enumerate(d['keystrokes']) if k[0]==key]
    for i in indexes:
        myplot.plotKeystroke(d, i)
p('Return')
p('Oemplus')